# Experimenting with our own Approach

## TODO:

- [x] import CORA dataset
- [ ] implement our approach: consider k-hop neighbors' input features and the current node's hidden state
- [ ] experiment a first model with GCN as the message passing scheme (later we'll experiment with GAT and GraphSAGE)
- [ ] track over-smoothing with **MAD** and **MADGap** over: 
    - [ ] K hops considered
    - [ ] epochs

#### Side notes

- Try to inverse the k-hop neighbors (start from remote and move down)
- Early stop for nodes that don't have k-hop neighbors (just pass the hidden state)

## Setup environment

- load packages from google drive (to install once)
- configure working directory (to download datasets)

In [2]:
# setup colab environment
import os, sys
import os.path as osp
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [3]:
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/utils.py

--2021-05-28 14:18:57--  https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1072 (1.0K) [text/plain]
Saving to: ‘data.py’

data.py             100%[===================>]   1.05K  --.-KB/s    in 0s      

2021-05-28 14:18:57 (72.8 MB/s) - ‘data.py’ saved [1072/1072]

--2021-05-28 14:18:57--  https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2333 (2.3K) [text/plain]
Saving to: ‘utils

In [5]:
# import utility functions
from data import load_dataset

In [7]:
path = osp.join(os.getcwd(), 'data')
cora_dataset = load_dataset(path, 'Cora')
G = cora_dataset[0] # only graph of the dataset

# MAD_cora = mean_average_distance(x=G.x)
# MADGap_cora = mean_average_distance_gap(x=G.x, adj_matrix=tg.utils.to_dense_adj(G.edge_index)[0])
# print(f'Initial MAD for Cora: {MAD_cora}')
# print(f'Initial MADGap for Cora: {MADGap_cora}')


# Our Approach

In [8]:
def xavier(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-2)))
        tensor.data.uniform_(-stdv, stdv)

def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)

In [9]:
def filter_neighbors(edge_index, num_nodes=None, k=1):
    # TODO: keep only k-hop neighbors
    return edge_index

In [ ]:
from torch_geometric.nn import MessagePassing

class KHopGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, K):
        self(KHopNet, self).__init__(aggr='add')
        self.lin = torch.nn.Linear(in_channels, out_channels)

        self.reset_parameters()
    
    def reset_parameters(self):
        xavier(self.lin.weight)
        zeros(self.lin.bias)
    
    def forward(self, x, h, edge_index, k):
        """
        Args:
            x (tensor): node input features, with shape [N, in_channels]
            h (tensor): node hidden states, with shape [N, in_channels]
            edge_index (tensor): graph edges, with shape [2, E]
            k (int): distance from the neighbors
        """
        # Step 1: filter edges to keep only k-hop neighbors
        edge_index = filter_neighbors(edge_index, num_nodes=x.size(0), k=k)

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        h = self.lin(h)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, h=h, norm=norm)
    
    def message(self, x_i, x_j):
        # x_j are input features of the neighbors

        # Step 4: Normalize node features.
        return nrom.view(-1, 1) * x_j
    
    def update(self, aggr_out, norm, h):
        # Step 5: add target node hidden state and return new node embeddings
        aggr_out += norm.view(-1, 1) * h
        return aggr_out